In [ ]:
import sys
sys.path.insert(0, '/share/project/yfl/codebase/git/ireg/src')
from pathlib import Path
from eval_utils.refcoco_utils import REFER
import json
from ipywidgets import interact,fixed
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon, Rectangle
import os.path as osp
import skimage.io as io
import cv2
import os

# Refer Dataset Tool

In [ ]:
refcoco_dir = Path('/share/project/yfl/database/REG_DATA/RefCOCO')
img_dir = Path('/share/project/yfl/database/REG_DATA/train2014/imgs')

dataset = 'refcoco+'
refcoco_dir = osp.join(refcoco_dir, dataset)
split_map = {'refcoco': 'unc', 'refcoco+': 'unc', 'refcocog': 'umd'}
refer = REFER(dataset=dataset,splitBy=split_map[dataset], img_dir=img_dir, ref_dir=refcoco_dir)

## Visualization for Comparing

In [ ]:
# 加载并过滤ireg的数据
with open('/share/project/yfl/codebase/git/ireg/misc/generated_results/IREG/refcoco+/Dialog_LAST/testB/testBresult.json', 'r') as fn:
    ireg = json.load(fn)

total = len(ireg)
cider = 0.
ofa_acc = 0.
original_cider = 0.
original_ofa_acc = 0.
ireg_filter_data = []
for itum in ireg:
    for i in range(5):
        last = itum['dialog_generate_sent_ofa_iou'][i]
        if i+1==5 or itum['dialog_generate_sent_ofa_iou'][i+1]==-1:
            break
    if last >  itum['dialog_generate_sent_ofa_iou'][0]:
        cider += itum['cider']
        ofa_acc += itum['last_round_ofa_iou']
        ireg_filter_data.append(itum)
    else:
        cider += itum['cider_first_round']
        ofa_acc += 1 if itum['dialog_generate_sent_ofa_iou'][0] else 0
    original_cider += itum['cider']
    original_ofa_acc += itum['last_round_ofa_iou']

print(cider/total, original_cider/total, ofa_acc/total, original_ofa_acc/total)

In [ ]:
ireg_filter_data

In [ ]:
with open('/share/project/yfl/codebase/git/ireg/misc/generated_results/referwhat/refcoco+/eval/pred_testB_2.json', 'r') as fn:
    d = json.load(fn)


referwhat = {}
for k,v in d.items():
    referwhat[k] = v['dialog']

In [ ]:
with open('/share/project/yfl/codebase/git/ireg/misc/generated_results/referwhat/refcoco+/eval/pred_testB_2.json', 'r') as fn:
    d2 = json.load(fn)
d2

In [ ]:
referwhat

In [ ]:
def draw_gen_text_for_dialog(ireg_data, referwhat_data, index):
    ref_id = ireg_data[index]['ref_id']
    ref = refer.Refs[ref_id]
    image = refer.Imgs[ref['image_id']]
    if 'train2014' in image['file_name']:
        img_path = os.path.join('/share/project/yfl/database/REG_DATA/train2014/imgs', image['file_name'])
    else:
        img_path = os.path.join('/share/project/yfl/database/REG_DATA/train2014/imgs', image['file_name'])
    img = cv2.imread(img_path)
    # ground truth
    ref_box = refer.refToAnn[ref_id]['bbox']
    ref_box = [int(x) for x in ref_box]
    cv2.rectangle(img, (ref_box[0],ref_box[1]), (ref_box[0]+ref_box[2],ref_box[1]+ref_box[3]), (0,255,0), 4)
    img = img[:,:,::-1]
    plt.figure(dpi=150)
    plt.imshow(img)
    plt.show()
    print("=======================IREG========================")
    for k,v in ireg_data[index].items():
        print("{} : {}".format(k,v))
    print("=======================Referwhat====================")
    print(referwhat_data[str(ref['ann_id'])])
    print("=======================GT REs====================")
    for i, s in enumerate(ref['sentences']):
        print(f'ref_{i}: {s["raw"]}')
    print(f'ref_id: {ref_id}')
    print(image['file_name'])

In [ ]:
len(ireg_filter_data)

In [ ]:
interact(draw_gen_text_for_dialog, ireg_data=fixed(ireg), referwhat_data=fixed(referwhat), index=list(range(300)))

In [ ]:
refcoco_dir = Path('/share/project/yfl/database/REG_DATA/RefCOCO')
img_dir = Path('/share/project/yfl/database/REG_DATA/train2014/imgs')

dataset = 'refcocog'
refcoco_dir = osp.join(refcoco_dir, dataset)
split_map = {'refcoco': 'unc', 'refcoco+': 'unc', 'refcocog': 'umd'}
refer2 = REFER(dataset=dataset,splitBy=split_map[dataset], img_dir=img_dir, ref_dir=refcoco_dir)

_ = refer2.getRefIds(split='train')
len(_)

In [ ]:
with open('/share/project/yfl/codebase/git/ireg/misc/ireg_data_collection/vlt5_ofa_scst_combine_clamp_mmi_refcocog_train_bad_sents.json', 'r' ) as fn:
    bad_sent =json.load(fn)

In [ ]:
ref_ids_set = set()
for itum in bad_sent:
    ref_ids_set.add(itum['ref_id'])

In [ ]:
len(ref_ids_set)

# Statistic for Dialog

In [ ]:
with open('../generate_result/refcoco+/vlt5_ofa_mmi_scst_combine_dialog_sp_training_threshold_0.5/testB/Noname.json', 'r') as f:
    datum = json.load(f)
    
one_round_datum = []
two_round_datum = []

for item in datum:
    if item['dialog_generate_sent'][1] == "":
        one_round_datum.append(item)
    else:
        two_round_datum.append(item)
        
iou_up = []
iou_down = []

cider_up = []
cider_down = []

iou_up_cider_down = []
iou_up_cider_up = []

for item in two_round_datum:
    if item['dialog_generate_sent_ofa_iou'][1] >= item['dialog_generate_sent_ofa_iou'][0]:
        iou_up.append(item)
    else:
        iou_down.append(item)
        
    if item['cider'] >= item['cider_first_round']:
        cider_up.append(item)
    else:
        cider_down.append(item)
        
    if item['dialog_generate_sent_ofa_iou'][1] >= item['dialog_generate_sent_ofa_iou'][0] and item['cider'] < item['cider_first_round']:
        iou_up_cider_down.append(item)
    elif item['dialog_generate_sent_ofa_iou'][1] >= item['dialog_generate_sent_ofa_iou'][0] and item['cider'] >= item['cider_first_round']:
        iou_up_cider_up.append(item)

In [ ]:
len(one_round_datum), len(two_round_datum), len(iou_up), len(cider_up), len(iou_up_cider_down)

In [ ]:
# Do Some statistical work

cider_first_round = 0
cider_second_round = 0

iou_first_round = 0
iou_second_round = 0

coverage_cnt = 0
total_cnt = 0

first_sent_length = 0
second_sent_length = 0


for item in two_round_datum:
    cider_first_round += item['cider_first_round']
    cider_second_round += item['cider']
    
    iou_first_round += item['dialog_generate_sent_ofa_iou'][0]
    iou_second_round += item['dialog_generate_sent_ofa_iou'][1]
    
    first_sent = item['dialog_generate_sent'][0].split(" ")
    second_sent = item['dialog_generate_sent'][1].split(" ")
    
    first_sent_length += len(first_sent)
    second_sent_length += len(second_sent)
    
    for word in first_sent:
        if word in second_sent:
            coverage_cnt += 1
    total_cnt += len(first_sent)

n = len(two_round_datum)
cider_first_round/n,cider_second_round/n,iou_first_round/n,iou_second_round/n,coverage_cnt/total_cnt, first_sent_length/n, second_sent_length/n

In [ ]:
cider_everage = 0
iou_everage = 0
for item in one_round_datum:
    
    cider_everage += item['cider']
    iou_everage += item['dialog_generate_sent_ofa_iou'][0]
m = len(one_round_datum)
cider_everage/m,iou_everage/m

# Visualization For Dialog

In [ ]:
# file_names = {
#     "epoch_BEST_last_round",
#     "epoch_BEST_dialog_result",
#     "epoch_BEST",
# }

# data = {}
# for file_name in file_names:
#     with open('./generate_result/refcoco+/vlt5_ofa_dialog/testB/'+ file_name+'.json','r') as f:
#         data[file_name] = json.load(f)
data = {}
reg_file_names = {
    "epoch_BEST_last_round",
    "epoch_BEST_dialog_result",
    "epoch_BEST",
    "epoch_BEST_zero_shot_result",
    "epoch_19",
    "epoch_19_lr_5e-05",
}
for file_name in reg_file_names:
    with open('./generate_result/refcoco+/REG/testB/'+file_name+'.json','r') as f:
        key = 'REG_'+file_name
        data[key] = json.load(f)

In [ ]:
path = '../generate_result/refcoco+/vlt5_ofa_dialog_sp_training_one_model_with_task1/testB/best_epoch.json'
with open(path, 'r') as f:
    data = json.load(f)

In [ ]:
import sys
import os.path as osp

if osp.join('/raid_sda/yfl/codebase/VL-T5-REG/VL-T5/src/', 'refer2/evaluation') not in sys.path:
    sys.path.insert(0, osp.join('/raid_sda/yfl/codebase/VL-T5-REG/VL-T5/src/', 'refer2/evaluation'))
    
from refEvaluation import RefEvaluation


first_round_pred = []
for i, item in enumerate(data):
    first_round_pred.append(
        {
            'ref_id': item['ref_id'],
            'sent': item['dialog_generate_sent'][0],
        }
    )

evaluator_first_round = RefEvaluation(refer, first_round_pred)
result = {}
CIDEr_sc_first_round, CIDEr_scs_first_round, METEOR_sc_first_round, METEOR_scs_first_round = evaluator_first_round.evaluate()
result['CIDEr_first_round'] = CIDEr_sc_first_round
result['CIDErs_first_round'] = CIDEr_scs_first_round
result['METEOR_first_round'] = METEOR_sc_first_round
result['METEORs_first_round'] = METEOR_scs_first_round
for i, item in enumerate(data):
    item['cider_first_round'] = result['CIDErs_first_round'][i]
    item['meteor_first_round'] = result['METEORs_first_round'][i]

In [ ]:
pred = []
for i, item in enumerate(data):
    first_round_pred.append(
        {
            'ref_id': item['ref_id'],
            'sent': item['sent'],
        }
    )

evaluator_first_round = RefEvaluation(refer, first_round_pred)
evaluator_first_round.evaluate()

In [ ]:
data

In [ ]:
round2 = []
for index,item in enumerate(data):
    if data[index]['dialog_generate_sent_ofa_iou'][1] != -1:
        round2.append(item)
        
print(len(data),len(round2))

up_up = 0
up_down = 0
down_up = 0
down_down = 0
equal = 0
for item in round2:
    iou = item['dialog_generate_sent_ofa_iou']
    cider = item['cider']
    cider_first_round = item['cider_first_round']
    if iou[1] > iou[0] and cider>cider_first_round:
        up_up += 1
    if iou[1] > iou[0] and cider<cider_first_round:
        up_down += 1
    if iou[1] < iou[0] and cider>cider_first_round:
        down_up += 1
    if iou[1] < iou[0] and cider<cider_first_round:
        down_down += 1
        
    if iou[1] == iou[0] and cider==cider_first_round:
        equal += 1
up_up,up_down,down_up,down_down,equal

In [ ]:
with open('/raid_sda/yfl/codebase/VL-T5-REG/VL-T5/src/generate_result/refcoco+/2022.10.19/testA/epoch13.json', 'r') as f:
    data = json.load(f)

In [ ]:
# The format of data is: 
#                         [{'refid':...,
#                          'dialog_generate_sent':...,
#                          ...
#                          },
#                          ...
#                         ]

def draw_gen_text_for_dialog(data, index):
    ref_id = data[index]['ref_id']
    ref = refer.Refs[ref_id]
    image = refer.Imgs[ref['image_id']]
    if 'train2014' in image['file_name']:
        img_path = os.path.join('/raid_sda/yfl/datasets/train2014', image['file_name'])
    else:
        img_path = os.path.join('/raid_sda/yfl/datasets/val2014', image['file_name'])
    img = cv2.imread(img_path)
    # ground truth
    ref_box = refer.refToAnn[ref_id]['bbox']
    ref_box = [int(x) for x in ref_box]
    cv2.rectangle(img, (ref_box[0],ref_box[1]), (ref_box[0]+ref_box[2],ref_box[1]+ref_box[3]), (0,255,0), 4)
    img = img[:,:,::-1]
    plt.figure(dpi=150)
    plt.imshow(img)
    plt.show()
    print("First round generate sent:{}".format(data[index]['dialog_generate_sent'][0]))
    print("First round cider:{:.3f} meteor:{:.3f} iou:{:.3f}".format(data[index]['cider_first_round'], data[index]['meteor_first_round'], data[index]['dialog_generate_sent_ofa_iou'][0]))
    print("Second round generate sent:{}".format(data[index]['sent']))
    print("Second round cider:{:.3f} meteor:{:.3f} iou:{:.3f}".format(data[index]['cider'], data[index]['meteor'], 
          data[index]['dialog_generate_sent_ofa_iou'][1]))
    for i, s in enumerate(ref['sentences']):
        print(f'ref_{i}: {s["raw"]}')
    print(f'ref_id: {ref_id}')
    print(image['file_name'])

In [ ]:
round2 = []
for index,item in enumerate(data):
    if data[index]['dialog_generate_sent_ofa_iou'][1] != -1:
        round2.append(item)

In [ ]:
interact(draw_gen_text_for_dialog, data=fixed(round2), index=list(range(100)))

# Statistic for Bad sent

In [ ]:
file_name = 'REG_mmi_refcocog_vlt5_bad_sent_threshold_0.5_with_bbox_val.json'
with open(file_name, 'r') as f:
    data = json.load(f)
    
len(data)

In [ ]:
ref_ids = []
refid2sent = {}
refid2box = {}
ids = set()
for item in data:
    ids.add(int(item['ref_id']))
    ref_ids.append(int(item['ref_id']))
    refid2sent[int(item['ref_id'])] = item['sent']
    refid2box[int(item['ref_id'])] = item['bbox']

# Visualization for Bad sent

In [ ]:
file_name = './new_generate_sent_set/vlt5_reg/refcoco/vlt5_reg_refcoco_bad_sent_threshold_0.5_with_bbox_testA.json'
with open(file_name, 'r') as f:
    data = json.load(f)
    
def draw(data, index):
    item = data[index]
    ref_id = item['ref_id']
    sent = item['sent']
    ofa_bbox = item['bbox']
    
    ref = refer.Refs[ref_id]
    image = refer.Imgs[ref['image_id']]
    if 'train2014' in image['file_name']:
        img_path = os.path.join('/raid_sda/yfl/datasets/train2014', image['file_name'])
    else:
        img_path = os.path.join('/raid_sda/yfl/datasets/val2014', image['file_name'])
    img = cv2.imread(img_path)
    # ground truth
    ref_box = refer.refToAnn[ref_id]['bbox']
    ref_box = [int(x) for x in ref_box]
    cv2.rectangle(img, (ref_box[0],ref_box[1]), (ref_box[0]+ref_box[2],ref_box[1]+ref_box[3]), (0,255,0), 4)
    cv2.rectangle(img, (int(ofa_bbox[0]),int(ofa_bbox[1])), (int(ofa_bbox[2]),int(ofa_bbox[3])), (0,0,255), 4)
    img = img[:,:,::-1]
    plt.figure(dpi=150)
    plt.imshow(img)
    plt.show()
    print("Generate Sentence:  {}".format(sent))
    print("GT Sentence:")
    for i, s in enumerate(ref['sentences']):
        print(f'\t ref_{i}: {s["raw"]}')
    print(f'\nref_id: {ref_id}')
    print(image['file_name'])    

In [ ]:
interact(draw, data=fixed(data), index=list(range(100)))

In [ ]:
file_name = './new_generate_sent_set/vlt5_reg/refcoco+/vlt5_reg_refcoco+_bad_sent_threshold_0.5_with_bbox.json'
with open(file_name, 'r') as f:
    train_data = json.load(f)

In [ ]:
interact(draw, data=fixed(train_data), index=list(range(100)))

In [ ]:
file_name = './generate_sent_set/REG_mmi/refcoco+/REG_mmi_refcoco+_vlt5_good_sent_threshold_0.5_with_bbox.json'
with open(file_name, 'r') as f:
    good_data = json.load(f)

# Original visualization

In [ ]:
def draw_gen_text(index):
    ref_id = data["REG_epoch_BEST"][index]['ref_id']
    ref = refer.Refs[ref_id]
    image = refer.Imgs[ref['image_id']]
    if 'train2014' in image['file_name']:
        img_path = os.path.join('/raid_sda/yfl/datasets/train2014', image['file_name'])
    else:
        img_path = os.path.join('/raid_sda/yfl/datasets/val2014', image['file_name'])
    img = cv2.imread(img_path)
    # ground truth
    ref_box = refer.refToAnn[ref_id]['bbox']
    ref_box = [int(x) for x in ref_box]
    cv2.rectangle(img, (ref_box[0],ref_box[1]), (ref_box[0]+ref_box[2],ref_box[1]+ref_box[3]), (0,255,0), 4)
    img = img[:,:,::-1]
    plt.figure(dpi=150)
    plt.imshow(img)
    plt.show()
    for file_name, d in data.items():
        text = d[index]['sent']
        print('{} gen:'.format(file_name), text)
        print('cider:', data[file_name][index]['cider'])
        print('meteor:', data[file_name][index]['meteor'])
        print('--------------------------')
    for i, s in enumerate(ref['sentences']):
        print(f'ref_{i}: {s["raw"]}')
    print(f'ref_id: {ref_id}')
    print(image['file_name'])

In [ ]:
interact(draw_gen_text, index=list(range(len(data["REG_epoch_BEST"]))))

# PalyGround for interact

In [ ]:
def f(data,index):
    return data[index]

datum = [5,8,9,0]
interact(f, data=fixed(datum), index=list(range(len(datum))))

# Some backup code

In [ ]:
# Old refer visualization

def draw_box_gen_text(index):
    ref = refer.Refs[data[index]['ref_id']]
    ax = plt.gca()
    # show image
    image = refer.Imgs[ref['image_id']]
    I = io.imread(osp.join(refer.IMAGE_DIR, image['file_name']))
    ax.imshow(I)
    ann_id = ref['ann_id']
    ann = refer.Anns[ann_id]
    bbox = 	refer.getRefBox(ref['ref_id'])
    box_plot = Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], fill=False, edgecolor='green', linewidth=3)
    ax.add_patch(box_plot)

	# show refer expression
    for sid, sent in enumerate(ref['sentences']):
        print('%s. %s' % (sid+1, sent['sent']))

    print('Gen RE: {}'.format(data[index]['sent']))